In [1]:
import torch
from torch import nn, optim
import torch.nn.functional as F

from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets, transforms
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import scanpy as sc
import plotly.express as px
import plotly.io as pio
import sklearn.preprocessing
import sklearn.model_selection




torch.autograd.set_detect_anomaly(True)

In [2]:
import platform
if platform.platform() == 'macOS-10.16-x86_64-i386-64bit':
    pio.renderers.default = 'notebook'
    device = torch.device('mps')
    print("Using Apple MPS on Macbook Pro")
    gmount = False
    
elif platform.platform() == 'Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic':
    pio.renderers.default = 'colab'
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print("Using CUDA GPU on Colab")
        gmount = True

Using Apple MPS on Macbook Pro


In [3]:
scdata = sc.read_h5ad("/Users/eamonmcandrew/Desktop/Single_cell_integration/Data/Multi-ome/GEX.h5ad")

In [4]:
scdata

AnnData object with n_obs × n_vars = 69249 × 13431
    obs: 'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker'
    var: 'feature_types', 'gene_id'
    uns: 'ATAC_gene_activity_var_names', 'dataset_id', 'genome', 'organism'
    obsm: 'ATAC_gene_activity', 'ATAC_lsi_full', 'ATAC_lsi_red', 'ATAC_umap', 'GEX_X_pca', 'GEX_X_umap'
    layers: 'counts'

In [5]:
def stratified_split(data, test_size, random_state, split_criteria):
    """
    Splits the data into train and test sets stratified by the batch column
    """
    train = []
    test = []
    for batch in data.obs[split_criteria].unique():
        batch_data = data[data.obs[split_criteria] == batch]
        batch_train, batch_test = sklearn.model_selection.train_test_split(batch_data, test_size=test_size, random_state=random_state)
        batch_train, batch_test = list(batch_train.obs.index), list(batch_test.obs.index)
        train.extend(batch_train)
        test.extend(batch_test)
        
    return train, test


In [6]:
train, test = stratified_split(scdata, 0.2, 9000, split_criteria='cell_type')

In [7]:
train_data = scdata[train]
test_data = scdata[test]

len(train_data), len(test_data)

(55392, 13857)

In [8]:
if gmount == True:
    from google.colab import drive
    drive.mount('/content/drive')
    path = '/content/drive/My Drive/Colab Notebooks/Experiments/' 
    scdata = sc.read_h5ad("/content/gdrive/MyDrive/scintegration/GEX.h5ad")

In [9]:
import wandb
wandb.login()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: eamomc. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/eamonmcandrew/.netrc
wandb: ERROR Unable to read ~/.netrc


True

In [10]:
# wandb.init(project="Single Cell Omics integration", entity="scintegration")

In [11]:
class GEX_Dataset(torch.utils.data.Dataset):
    
      def __init__(self, data,  scaler = None, cat_var = None, label_encoder =None):
          
            self.data = data
            
            # we need to work with the dense matrix
            self.values = data.X.todense()
            
            self.cat_var = cat_var
            
            if label_encoder == "numeric":
            # numerically encode the labels
              cat_var_data =  torch.tensor(sklearn.preprocessing.LabelEncoder().fit_transform(self.data.obs[self.cat_var]), dtype = torch.long)
            
            elif label_encoder == "range_map":
              cat_var_data =  sklearn.preprocessing.LabelEncoder().fit_transform(self.data.obs[self.cat_var])
              cat_var_data = cat_var_data.reshape(-1, 1) 
              cat_var_data = torch.tensor(sklearn.preprocessing.MinMaxScaler().fit_transform(cat_var_data), dtype = torch.float32)

            elif label_encoder == "one_hot":
              cat_var_data =  torch.tensor(sklearn.preprocessing.LabelEncoder().fit_transform(self.data.obs[self.cat_var]))
              cat_var_data = cat_var_data.reshape(-1, 1)
              cat_var_data = sklearn.preprocessing.OneHotEncoder().fit_transform(cat_var_data).toarray()
              cat_var_data = torch.tensor(cat_var_data, dtype=torch.float32)
              

            self.cat_var_data = torch.tensor(cat_var_data)
            
            # scale the data according to user inpt to scaler argument
            if scaler == "Standard":
                self.scaled_values = torch.tensor(sklearn.preprocessing.StandardScaler().fit_transform(self.values), dtype = torch.float32)
            elif scaler == "MinMax":
                self.scaled_values = torch.tensor(sklearn.preprocessing.MinMaxScaler().fit_transform(self.values),  dtype = torch.float32)
            else:
                self.scaled_values = torch.tensor(self.values, dtype = torch.float32)
                
    #   return the number of genes when called 
             
      @property
      def n_features(self):
          return self.values.shape[1]

      @property
      def n_catagories(self):
          return self.cat_var_data.shape[1]
  
          
    #  A dataset class needs the following two methods to work with the dataloader class     
          
    #   return the number of cells when called
      def __len__(self):
          return len(self.data)
    
    #  return an individual cell and its label when called
      def __getitem__(self, idx):
           return self.scaled_values[idx], self.cat_var_data[idx]

    

In [12]:
GEX_Dataset_train = GEX_Dataset(train_data, scaler = "Standard", cat_var = "batch", label_encoder = "one_hot")

GEX_Dataset_test = GEX_Dataset(test_data, scaler = "Standard", cat_var = "batch", label_encoder = "one_hot")


/var/folders/mw/_f0gy0f54t739qb79y8cxb_40000gn/T/ipykernel_29778/612472411.py:30: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

/Users/eamonmcandrew/opt/anaconda3/envs/scINTEGRATION/lib/python3.9/site-packages/sklearn/utils/validation.py:727: FutureWarning:

np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html

/Users/eamonmcandrew/opt/anaconda3/envs/scINTEGRATION/lib/python3.9/site-packages/sklearn/utils/validation.py:727: FutureWarning:

np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html

/var/folders/mw/_f0gy0f54t73

In [13]:
input_size = GEX_Dataset_train.n_features
output_size = GEX_Dataset_train.n_catagories


# batch_size = 256
# epochs = 30
# lr = 1e-4
# dropout = 0.2



# wandb.config = {
#   "learning_rate": lr,
#   "epochs": epochs,
#   "batch_size": batch_size,
#   "dropout": dropout,
# }

log_interval = 100

In [14]:
output_size


13

In [15]:
class classifier(nn.Module):
    def __init__(self, dropout, hidden_size):
        super(classifier, self).__init__()
        self.cfc1 = nn.Linear(input_size, hidden_size)
        self.dropout = nn.Dropout(dropout)
        self.cfc2 = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        x = self.cfc1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.cfc2(x)
        x = F.softmax(x, dim = 1)
        return(x)

In [16]:
# GEX_dataloader_train = torch.utils.data.DataLoader(GEX_Dataset_train, batch_size = wandb.config.batch_size, shuffle = True)
# GEX_dataloader_test = torch.utils.data.DataLoader(GEX_Dataset_test, batch_size = wandb.config.batch_size, shuffle = True)
# model = classifier()
# model.to(device)
# optimizer = optim.Adam(model.parameters(), lr=wandb.config.lr)
# model.train()

# criterion = nn.CrossEntropyLoss()

In [17]:
def train_one_epoch(epoch, GEX_dataloader_train, model, optimizer, criterion):
    # Set the model to train mode
    model.train()
    
    # Initialize lists to store the losses and accuracies for each batch
    epoch_loss_list = []
    epoch_accuracy_list = []
    
    # Iterate over the batches in the dataloader
    for batch_idx, (data, target) in enumerate(GEX_dataloader_train):
        # Move the data and target tensors to the specified device (GPU)
        data, target = data.to(device), target.to(device)
        # Clear the gradients of all optimized parameters
        optimizer.zero_grad()
        # Feed the data through the model and get the output
        output = model(data)
        # Calculate the loss using the specified loss function
        loss = criterion(output, target)
        # Calculate the accuracy by comparing the model's predictions to the ground truth labels
        accuracy = (output.argmax(1) == target.argmax(1)).type(torch.float).mean().item()
        # Backpropagate the loss to update the model's parameters
        loss.backward()
        # Update the model's parameters using the optimizer
        optimizer.step()
        # Append the loss and accuracy for this batch to the corresponding lists
        epoch_loss_list.append(loss.item())
        epoch_accuracy_list.append(accuracy)
        
    # Calculate the mean loss and accuracy for the entire epoch
    epoch_loss = np.mean(epoch_loss_list)
    epoch_accuracy = np.mean(epoch_accuracy_list)
    # Log the epoch loss and accuracy
    wandb.log({"Train epoch loss": epoch_loss})
    wandb.log({"Train epoch accuracy": epoch_accuracy})
    
    # Return the epoch loss and accuracy
    return epoch_loss, epoch_accuracy

            
        

In [18]:
def evaluate_one_epoch(epoch, GEX_Dataset_test, model, optimizer, criterion):
    # Set the model to eval mode
    model.eval()
    # Tell PyTorch not to track gradients while evaluating the model
    with torch.no_grad():
        # Initialize lists to store the losses and accuracies for each batch
        epoch_loss_list = []
        epoch_accuracy_list = []
        
        # Iterate over the batches in the dataloader
        for batch_idx, (data, target) in enumerate(GEX_Dataset_test):
            # Move the data and target tensors to the specified device (GPU)
            data, target = data.to(device), target.to(device)
            # Feed the data through the model and get the output
            output = model(data)
            # Calculate the loss using the specified loss function
            loss = criterion(output, target)
            # Calculate the accuracy by comparing the model's predictions to the ground truth labels
            accuracy = (output.argmax(1) == target.argmax(1)).type(torch.float).mean().item()
            # Append the loss and accuracy for this batch to the corresponding lists
            epoch_loss_list.append(loss.item())
            epoch_accuracy_list.append(accuracy)
            # Log the batch loss and accuracy
            wandb.log({"Val loss": loss.item()})
            wandb.log({"val accuracy": accuracy})
            
            # Calculate and log the confusion matrix for this batch
            # ground_truth_class_ids = target.argmax(1).cpu().numpy()
            # predicted_class_ids = output.argmax(1).cpu().numpy()
            # wandb.log({"conf_mat" : wandb.plot.confusion_matrix(probs=None, y_true=ground_truth_class_ids, preds=predicted_class_ids, class_names=scdata.obs["batch"].unique())})
        
        # Calculate the mean loss and accuracy for the entire epoch
        epoch_loss = np.mean(epoch_loss_list)
        epoch_accuracy = np.mean(epoch_accuracy_list)
        # Log the epoch loss and accuracy
        wandb.log({"Validation epoch loss": epoch_loss})
        wandb.log({"Validation epoch accuracy": epoch_accuracy})
        
    # Return the epoch loss and accuracy
    return epoch_loss, epoch_accuracy


In [19]:
sweep_configuration = {
    'method': 'bayes',
    'name': 'sweep',
    'metric': {
        'goal': 'maximize', 
        'name': 'val_acc'
		},
    'parameters': {
        'batch_size': {'values': [128, 256, 512]},
        'epochs': {'value' : 100},
        'lr': {'max': 0.01, 'min': 0.0001},
        'random_seed': {'value': 9000},
        'dropout': {'value': 0.2},
        'hidden_size': {'values': [ 5, 10, 20, 30]}
        
}}

sweep_id = wandb.sweep(sweep_configuration, project="Single Cell Omics integration", entity="scintegration")

Create sweep with ID: o931h183
Sweep URL: https://wandb.ai/scintegration/Single%20Cell%20Omics%20integration/sweeps/o931h183


In [20]:
def train_func():
    # Initialize a new run in Weights & Biases (wandb)
    run = wandb.init(project="Single Cell Omics integration", entity="scintegration")

    # Load the learning rate, batch size, epochs, random seed, dropout, and hidden size from the wandb configuration
    lr = wandb.config.lr
    batch_size = wandb.config.batch_size
    epochs = wandb.config.epochs
    random_seed = wandb.config.random_seed
    dropout = wandb.config.dropout
    hidden_size = wandb.config.hidden_size

    # Instantiate the model, optimizer, and criterion outside the for loop
    model = classifier(dropout=dropout, hidden_size=hidden_size)

    # Move the model to the specified device (e.g. GPU)
    model.to(device)

    # Instantiate the Adam optimizer with the specified learning rate
    optimizer = optim.Adam(model.parameters(), lr=wandb.config.lr)

    # Instantiate the Cross Entropy loss function
    criterion = nn.CrossEntropyLoss()
    
    # Create dataloaders for the training and test datasets
    GEX_dataloader_train = torch.utils.data.DataLoader(GEX_Dataset_train, batch_size = wandb.config.batch_size, shuffle = True)
    GEX_dataloader_test = torch.utils.data.DataLoader(GEX_Dataset_test, batch_size = wandb.config.batch_size, shuffle = True)

    for epoch in range(1, epochs + 1):
        # Train the model for one epoch
        train_loss, train_acc = train_one_epoch(epoch, GEX_dataloader_train, model, optimizer, criterion)

        # Evaluate the model on the test dataset
        val_loss, val_acc = evaluate_one_epoch(epoch, GEX_dataloader_test, model, optimizer, criterion)

        # Log the epoch, train accuracy, train loss, validation accuracy, and validation loss to wandb
        wandb.log({
          'epoch': epoch, 
          'train_acc': train_acc,
          'train_loss': train_loss, 
          'val_acc': val_acc, 
          'val_loss': val_loss
        })


In [21]:
wandb.agent(sweep_id, train_func, count=30)


wandb: Agent Starting Run: hg14f4vb with config:
wandb: 	batch_size: 256
wandb: 	dropout: 0.2
wandb: 	epochs: 100
wandb: 	hidden_size: 20
wandb: 	lr: 0.008855567795104852
wandb: 	random_seed: 9000
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: eamomc (scintegration). Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Train epoch accuracy,▁▅▆▆▆▆▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇█▇███████████████
Train epoch loss,█▄▃▃▃▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Val loss,▅▃▇▃█▅▅▆▃▃▄▄▃▃▃▃▄▁▃▄▃▃▃▃▃▃▃▁▃▃▁▃▅▂▂▄▂▁▂▃
Validation epoch accuracy,▁▂▂▃▃▃▄▃▃▄▄▄▄▄▂▅▇▇▄▆▆▇▇▇▇▇▆████▇████▇██▇
Validation epoch loss,█▇▇▆▆▆▅▆▆▅▅▅▅▅▇▄▂▂▅▃▃▂▂▂▂▁▃▁▁▁▁▂▁▁▁▁▂▁▁▂
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_acc,▁▅▆▆▆▆▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇█▇███████████████
train_loss,█▄▃▃▃▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val accuracy,▄▆▂▆▁▄▄▃▆▆▅▅▆▆▆▆▅█▆▅▆▆▆▆▆▆▆█▇▆█▇▄▇▇▅▇█▇▆
val_acc,▁▂▂▃▃▃▄▃▃▄▄▄▄▄▂▅▇▇▄▆▆▇▇▇▇▇▆████▇████▇██▇
val_loss,█▇▇▆▆▆▅▆▆▅▅▅▅▅▇▄▂▂▅▃▃▂▂▂▂▁▃▁▁▁▁▂▁▁▁▁▂▁▁▂


wandb: Agent Starting Run: 7ewweddp with config:
wandb: 	batch_size: 512
wandb: 	dropout: 0.2
wandb: 	epochs: 100
wandb: 	hidden_size: 10
wandb: 	lr: 0.008184489187500602
wandb: 	random_seed: 9000
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Train epoch accuracy,▁▅▆▇▇▇▇▇▇███████████████████████████████
Train epoch loss,█▄▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Val loss,▇▅▄█▄▅▂▃▃▄▄▂▃▂▂▁▂▃▃▇▃▂▃▂▂▂▁▃▁▃▅▂▃▃▄▂▂▃▅▃
Validation epoch accuracy,▁▄▆▇▇▇█▇█▇▇▇▇▇▇▇▇▇▇▇▇▇███████▇▇█▇█▇▇█▇██
Validation epoch loss,█▅▃▂▂▂▁▂▁▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▂▁▂▁▁▂▁▂▁▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_acc,▁▅▆▇▇▇▇▇▇███████████████████████████████
train_loss,█▄▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val accuracy,▂▄▅▁▅▄▇▆▆▅▆▇▆▇▇█▇▇▆▃▆▇▆▇▇▇█▆█▆▅▇▆▆▅▇▇▆▄▆
val_acc,▁▄▆▇▇▇█▇█▇▇▇▇▇▇▇▇▇▇▇▇▇███████▇▇█▇█▇▇█▇██
val_loss,█▅▃▂▂▂▁▂▁▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▂▁▂▁▁▂▁▂▁▁


wandb: Agent Starting Run: z4ynm1vf with config:
wandb: 	batch_size: 128
wandb: 	dropout: 0.2
wandb: 	epochs: 100
wandb: 	hidden_size: 5
wandb: 	lr: 0.007052917367459953
wandb: 	random_seed: 9000
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


...

wandb: Agent Starting Run: vzrkj6pb with config:
wandb: 	batch_size: 256
wandb: 	dropout: 0.2
wandb: 	epochs: 100
wandb: 	hidden_size: 10
wandb: 	lr: 0.00705005374364317
wandb: 	random_seed: 9000
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Train epoch accuracy,▁▃▄▄▄▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▆▇▇▇▇▇██████
Train epoch loss,█▆▅▅▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▃▂▂▂▂▂▁▁▁▁▁▁
Val loss,▆▇▆▇▆▇▆▇▇█▇▆▆▆▅▆▆▇▆▇█▇█▆▆▆▆▆▄▄▄▃▃▄▁▂▂▃▂▁
Validation epoch accuracy,▁▂▃▃▄▄▄▃▄▃▄▄▄▄▄▃▄▃▄▄▄▄▄▄▄▄▄▄▆▆▆▇▇▆██████
Validation epoch loss,█▇▆▆▅▅▅▆▅▆▅▅▅▅▅▅▅▆▅▅▅▅▅▅▅▅▅▅▃▃▂▂▂▃▁▁▁▁▁▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_acc,▁▃▄▄▄▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▆▇▇▇▇▇██████
train_loss,█▆▅▅▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▃▂▂▂▂▂▁▁▁▁▁▁
val accuracy,▂▁▃▂▃▂▃▂▂▁▂▃▃▃▄▃▃▂▃▂▁▂▁▃▃▃▃▃▅▅▅▆▆▅█▇▇▆▇█
val_acc,▁▂▃▃▄▄▄▃▄▃▄▄▄▄▄▃▄▃▄▄▄▄▄▄▄▄▄▄▆▆▆▇▇▆██████
val_loss,█▇▆▆▅▅▅▆▅▆▅▅▅▅▅▅▅▆▅▅▅▅▅▅▅▅▅▅▃▃▂▂▂▃▁▁▁▁▁▁


wandb: Agent Starting Run: f6v61uof with config:
wandb: 	batch_size: 256
wandb: 	dropout: 0.2
wandb: 	epochs: 100
wandb: 	hidden_size: 10
wandb: 	lr: 0.009219957964534124
wandb: 	random_seed: 9000
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Train epoch accuracy,▁▅▆▆▆▆▆▆▇▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▆▇▇▇▇▇███▇███
Train epoch loss,█▄▃▃▃▃▃▃▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▃▂▂▂▂▂▁▁▁▂▁▁▁
Val loss,▄▆▇▆▅▃▆▅█▅▄▃▅▄█▄▄▄▅▅▇▅█▄▄▂▅▆▅▆▅▄▆▁▄▅▅▆▃▂
Validation epoch accuracy,▁▄▃▃▄▄▄▄▅▅▅▅▄▃▄▄▄▄▅▄▅▅▄▄▄▅▄▅▄▄▃▄▆▇▇▇▇▇█▇
Validation epoch loss,█▅▅▅▅▅▅▅▄▄▄▄▄▅▅▅▅▅▄▄▄▄▅▄▅▄▅▄▅▅▆▄▂▂▂▂▂▂▁▂
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_acc,▁▅▆▆▆▆▆▆▇▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▆▇▇▇▇▇███▇███
train_loss,█▄▃▃▃▃▃▃▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▃▂▂▂▂▂▁▁▁▂▁▁▁
val accuracy,▅▃▂▃▄▆▃▄▁▄▅▆▄▅▁▅▅▅▄▄▂▄▁▅▅▇▄▃▄▃▄▅▃█▅▄▄▃▆▇
val_acc,▁▄▃▃▄▄▄▄▅▅▅▅▄▃▄▄▄▄▅▄▅▅▄▄▄▅▄▅▄▄▃▄▆▇▇▇▇▇█▇
val_loss,█▅▅▅▅▅▅▅▄▄▄▄▄▅▅▅▅▅▄▄▄▄▅▄▅▄▅▄▅▅▆▄▂▂▂▂▂▂▁▂


wandb: Agent Starting Run: i45mp957 with config:
wandb: 	batch_size: 128
wandb: 	dropout: 0.2
wandb: 	epochs: 100
wandb: 	hidden_size: 10
wandb: 	lr: 0.00827488676799856
wandb: 	random_seed: 9000
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


In [ ]:
# loss_list_train = []
# accuracy_list_train = []
# mean_loss = 0
# mean_accuracy =0

# for epoch in range(epochs+1):
#     epoch_loss_train = []
#     epoch_accuracy_train = []
#     for batch_idx, (data, target) in enumerate(GEX_Dataset_train):
#         data = data.to(device)
#         target = target.to(device)
#         optimizer.zero_grad()
#         output = model(data)
#         loss_train = criterion(output, target)
#         wandb.log({"training loss": loss_train})
#         loss_train.backward()
#         loss_list_train.append(loss_train.item())
#         optimizer.step()
#         accuracy_train = (output.argmax(1) == target.argmax(1)).type(torch.float).mean().item()
#         wandb.log({"Training accuracy": accuracy_train})
#         accuracy_list_train.append(accuracy_train)
#         if batch_idx % log_interval == 0:
#             print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(GEX_dataloader.dataset)} ({100. * batch_idx / len(GEX_dataloader):.0f}%)]\tLoss: {loss_train.item():.6f}, Accuracy: {accuracy_train:.2f}')
            
#     mean_epoch_loss_train = np.mean(epoch_loss_train)
#     mean_epoch_accuracy_train = np.mean(epoch_accuracy_train)
#     print(f'Epoch: {epoch}, Epoch Mean Loss: {mean_epoch_loss_train:.6f}, Epoch Mean Accuracy: {mean_epoch_accuracy_train:.2f}')
#     wandb.log({"epoch_loss": mean_epoch_loss_train})
#     wandb.log({"epoch_accuracy": mean_epoch_accuracy_train})
#     wandb.log({"epoch": epoch})

    